In [ ]:
from pyray import *
import random

# ----- Constants -----
PLAYER_Y = 420
PLAYER_WIDTH = 80  # Increased width to make gameplay easier
PLAYER_HEIGHT = 10
PLAYER_SPEED = 8

BALL_SIZE = 20
BALL_SPEED = 3

SCREEN_WIDTH = 800
SCREEN_HEIGHT = 450

BLOCK_WIDTH = 50
BLOCK_HEIGHT = 20
BLOCK_ROWS = 5
BLOCK_COLUMNS = SCREEN_WIDTH // BLOCK_WIDTH

COLORS = [DARKGRAY,
          MAROON,
          ORANGE,
          DARKGREEN,
          DARKBLUE,
          DARKPURPLE,
          DARKBROWN,
          GRAY,
          RED,
          GOLD,
          LIME,
          BLUE,
          VIOLET,
          BROWN,
          LIGHTGRAY,
          PINK,
          YELLOW,
          GREEN,
          SKYBLUE,
          PURPLE,
          BEIGE]
# ---------------------

class Block:
    def __init__(self, x, y, width, height, color):
        self.__x = x
        self.__y = y
        self.__width = width
        self.__height = height
        self.__color = color

    def draw(self):
        draw_rectangle(self.__x, self.__y, self.__width, self.__height, self.__color)

    def get_rectangle(self):
        return Rectangle(self.__x, self.__y, self.__width, self.__height)

def update_game_state(player_pos, ball_x, ball_y, ball_dir_x, ball_dir_y, blocks):
    # Restart game if "R" is pressed or ball is out of bounds
    if is_key_pressed(KeyboardKey.KEY_R) or ball_y > SCREEN_HEIGHT:
        player_pos = 400
        ball_x = SCREEN_WIDTH // 2
        ball_y = SCREEN_HEIGHT // 2
        ball_dir_x = random.choice([-1, 1])
        ball_dir_y = 1
        blocks.clear()
        for row in range(BLOCK_ROWS):
            for col in range(BLOCK_COLUMNS):
                x = col * BLOCK_WIDTH
                y = row * BLOCK_HEIGHT
                color = random.choice(COLORS)
                blocks.append(Block(x, y, BLOCK_WIDTH, BLOCK_HEIGHT, color))
    
    # Update player position
    if is_key_down(KeyboardKey.KEY_LEFT) and player_pos > 0:
        player_pos -= PLAYER_SPEED
    if is_key_down(KeyboardKey.KEY_RIGHT) and player_pos < SCREEN_WIDTH - PLAYER_WIDTH:
        player_pos += PLAYER_SPEED

    # Update ball position
    ball_x += ball_dir_x * BALL_SPEED
    ball_y += ball_dir_y * BALL_SPEED

    # Ball collisions with the player and wall
    if check_collision_recs(
        Rectangle(player_pos, PLAYER_Y, PLAYER_WIDTH, PLAYER_HEIGHT), 
        Rectangle(ball_x, ball_y, BALL_SIZE, BALL_SIZE)):
        ball_dir_y = -1
    elif ball_y <= 0:
        ball_dir_y = 1

    if ball_x <= 0:
        ball_dir_x = 1
    elif ball_x >= SCREEN_WIDTH - BALL_SIZE:
        ball_dir_x = -1

    # Ball collisions with blocks
    block_idx = 0
    while block_idx < len(blocks):
        block = blocks[block_idx]
        if check_collision_recs(
            block.get_rectangle(), 
            Rectangle(ball_x, ball_y, BALL_SIZE, BALL_SIZE)):
            del blocks[block_idx]
            ball_dir_y *= -1
        else:
            block_idx += 1

    return player_pos, ball_x, ball_y, ball_dir_x, ball_dir_y

def main():
    init_window(SCREEN_WIDTH, SCREEN_HEIGHT, "Breakout")
    set_target_fps(60)

    # ----- Initialize player and ball -----
    player_pos = 400

    ball_x = SCREEN_WIDTH // 2
    ball_y = SCREEN_HEIGHT // 2
    ball_dir_x = random.choice([-1, 1])
    ball_dir_y = 1
    # --------------------------------------
    
    # ----- Initialize blocks -----
    blocks = []
    for row in range(BLOCK_ROWS):
        for col in range(BLOCK_COLUMNS):
            x = col * BLOCK_WIDTH
            y = row * BLOCK_HEIGHT
            color = random.choice(COLORS)
            blocks.append(Block(x, y, BLOCK_WIDTH, BLOCK_HEIGHT, color))
    # -----------------------------

    running = True
    while running:
        if window_should_close():
            running = False
            break

        player_pos, ball_x, ball_y, ball_dir_x, ball_dir_y = update_game_state(player_pos, ball_x, ball_y, ball_dir_x, ball_dir_y, blocks)

        # Drawing
        begin_drawing()
        clear_background(WHITE)

        draw_rectangle(player_pos, PLAYER_Y, PLAYER_WIDTH, PLAYER_HEIGHT, BLACK)
        draw_rectangle(ball_x, ball_y, BALL_SIZE, BALL_SIZE, BLUE)
        for block in blocks:
            block.draw()

        end_drawing()

    close_window()

if __name__ == "__main__":
    main()
